Sample SLSN search based on initial conesearch. 

In [ ]:
import requests, os

In [ ]:
# This is the archive token which can be obtained from https://ampel.zeuthen.desy.de/live/dashboard/tokens
# In order to retrieve ZTF partnership alerts your token needs to have the appropriate access
token = os.environ["ARCHIVE_TOKEN"]   # I have mine stored
header = {"Authorization": "bearer "+token}

In [ ]:
workdir = './ampel_slsn'   # Assuming this exists

## 1. Creating archive alert stream based on alert properties. 

The pipeline will parse a set of alerts retrieved from the DESY/Ampel alert archive. This demonstration uses a limited consearch centered on a known SLSN (ZTF19abfvnns). This query can include filtering based on properties of the ZTF alert candidate field. For more options, see the `ztf.alert.candidate` section of https://zwickytransientfacility.github.io/ztf-avro-alert/schema.html

The obtained token (`resume_token`) will be used when configuring the alert loader below, and will allow the alert processor to operate on a stream of archive alerts. 

In [ ]:
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?'

In [ ]:
query = {
"cone": {
#    "ra": 193.28143551564065,
#    "dec": 33.488720966906016,
    "ra": 1.1691247,      
    "dec": 42.8698038,
    "radius": 2. / 60
  },
"candidate": {
    "rb": {
      "$gt": 0.3
    },
    "ndethist": {
      "$gt": 5,
      "$lte": 10000
    },
    "isdiffpos": {"$in": ["t", "1"]},
  }
}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

In [ ]:
if not response.ok:
    print( 'Query creation failed.')

In [ ]:
# The full response contains the resume token as well as the chunk size, i.e.
# how many alerts will be return in each call to the alert iterator.
response.json()

In [ ]:
resume_token = response.json()['resume_token']

At this point the alert archive will start the process of staging alerts for release. This process takes a few min (length depending on query size), during which time the resume_token will stay locked. 

## 2. Configure an alert filter

Only alerts passing these criteria will be propagated to full analysis. This partially replicates critera from the query filter implemented above, as the filter defined here could be used also for a live application where no initial archive selection is done. 


In [ ]:
from ampel.log.AmpelLogger import AmpelLogger
from ampel.contrib.hu.t0.RcfFilter import RcfFilter
logger = AmpelLogger.get_logger()

In [ ]:
# Decent filter parameters
filter_config = {
    "min_ndet": 6,          # Min number of detections
    "max_magpsf": 19.3,      
    "min_dist_to_sso": 15,     
    "min_gal_lat": 7,      
    "min_age": 10,
    "max_ipac_age": 100,
    "min_rb": 0.3,
#    "min_drb": 0.99,
}

In [ ]:
t0filter = RcfFilter( **filter_config, logger=logger )
t0filter.post_init()

## 3. Initiate DB connection and AMPEL context

Will create a local MongoDB collection `ztf_slsn` - if one exists this will be purged initially.


In [ ]:
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext

In [ ]:
# Create a vault - the way to input "secrets" into the system
secrets = { 
            "ztf/archive/token": token,
          }
vault = AmpelVault([DictSecretProvider(secrets)])

In [ ]:
# Assume that a configuration file exists here (see Ampel-HU-astro install instructions)
AMPEL_CONF = '../ampel_conf.yaml'

In [ ]:
channel = 'myslsn'

In [ ]:
# Assuming a local MongoDB instance
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'ztf_slsn',
    purge_db = True,
    vault = vault
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

## 4. Configure modules to be used in workflow

As the last step these are combined into the `directives` which are supplied to the alert processor.  

In [ ]:
# Catalogs to match positions to 
cat_conf = {
    'catalogs' : {
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        },
        'LSPhotoZZou' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['photoz','ra','dec','e_photoz','specz','_6','logMassBest','logMassInf','logMassSup'],
            'pre_filter' : None,
            'post_filter' : None,
            'all': False,
        },
        'wiseScosPhotoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto_Corr','ra','dec','wiseID','w1mCorr','w2mCorr'],
            'pre_filter' : None,
            'post_filter' : None,
        },
        'twoMPZ' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto','ra','dec','zSpec'],
            'pre_filter' : None,
            'post_filter' : None,
        },
         'PS1_photoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['raMean','decMean','z_phot','z_photErr', 'z_phot0', '_2'],
            'pre_filter' : None,
            'post_filter' : None,
        },
    }
}

In [ ]:
# New, new, also with all categories?
ampelz_conf = {
    "max_redshift_category" : 7,
    "t2_dependency": [
        {
            "unit": "T2CatalogMatch",
            "config": cat_conf,
            "link_override": {
                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
                }
        },
    ]
}

In [ ]:
# T2PhaseLimit: Determine fit region 
phaselimit_conf = {
    "half_time": 100,
    "plot_suffix": "png",
    "plot_dir": workdir,
    "max_flux": 2000,
    "min_sigma": 3,
    "risedec_fractions": [0.5,0.9],
    "tabulator": [
        {"unit": "ZTFT2Tabulator"}
    ],
    
}

In [ ]:
sncosmo_conf = {
    "sncosmo_model_name" : "salt2",
    "redshift_kind" : 'T2DigestRedshifts',
    "max_ampelz_group" : 7,      # For this purpose we use any available redshift
    "t2_dependency": [
        {
            "unit": "T2DigestRedshifts",
            "config": ampelz_conf,
        },
    ], 
    "tabulator": [
        {"unit": "ZTFT2Tabulator"}
    ],
    "plot_props": {
        "tags": ["SALT", "SNCOSMO"],
        "file_name": {
            "format_str": "%s_%s_%s.svg",
            "arg_keys": ["stock", "model", "redshift_kind"]
        },
        "title": {
            "format_str": "%s %s %s",
            "arg_keys": ["stock", "model", "redshift_kind"]
        },
        "fig_text": {
            "format_str": "%s %s \nz-source %s \nchisq %.2f ndof %s",
            "arg_keys": ["stock", "model", "redshift_kind", "chisq", "ndof"]
        },
        "width": 10,
        "height": 6,
        "id_mapper": "ZTFIdMapper",
        "disk_save": workdir,
    }
}

In [ ]:
directives = [
        {
            "channel": channel,
            "filter": {
                "unit": "RcfFilter",
                "config": filter_config,
                "on_stock_match": 'bypass',
            },
            "ingest": {
                "mux": {
                    "unit": "ZiArchiveMuxer",
                    "config": {
                        "history_days": 999,
                        "future_days": 999
                    },
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                            "state_t2": [
                                {
                                    "unit": "T2DigestRedshifts",
                                    "config": ampelz_conf,
                                },
                                {
                                    "unit": "T2MatchBTS",
                                },
                                {
                                    "unit": "T2PhaseLimit",
                                    "config": phaselimit_conf,
                                },
                                {
                                    "unit": "T2RunSncosmo",
                                    "config": sncosmo_conf,
                                },                            
                            ]
                        }
                    ],
                    "insert": {
                        "point_t2": [
                            {
                                'unit': 'T2CatalogMatch',
                                'config': cat_conf,
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                            },
                        ],
                    }

                }
            }
        }
    ]

## 5. Initiate an alert processor and process alerts

In [ ]:
ITER_MAX = 1000

In [ ]:
loader_config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", "stream":resume_token}

In [ ]:
ac = ctx.new_context_unit(
    unit = "AlertConsumer",
    process_name = "AP_test",
    iter_max = ITER_MAX,
    log_profile = os.environ.get('log_profile', 'debug'),
    shaper = "ZiDataPointShaper",
    compiler_opts = "ZiCompilerOptions",
    supplier = {
        "unit": "ZiAlertSupplier",
        'config': {
            'deserialize': None,
            'loader': {
                'unit': 'ZTFArchiveAlertLoader',
                'config': loader_config
            }
        }
    },
    directives = directives
)


In [ ]:
n = ac.run()

In [ ]:
print(f"{n} alerts processed for channel {channel}")

## 6. Process T2 tickets 

A number of alerts passed the filter criteria. For these, the additional module computations configured above were scheduled for computation. We will now carry out these operations. 

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_test',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()

## 7. Display out put figures

Some of the executed units produced debug figures. These are display here (assuming the hard-coded paths work).


In [ ]:
from IPython.core.display import SVG
import matplotlib.pyplot as plt
import matplotlib.image as pim

In [ ]:
# Phaselimit analysis of lc duration and peak
im = pim.imread(workdir+"/t2phaselimit_232711938.png")
plt.imshow(im)

In [ ]:
# SALT2 (SNIa) model fit to the data, using the redshift found from catalogs
SVG(filename=workdir+"/ZTF19abfvnns_salt2_T2DigestRedshifts.svg")

## 8. Store output

T3 units parses the full DB content. The sample unit prints selected properties to an external file. 

In [ ]:
table_config =  {
                                        'table_schema' : {
                                            'T2DigestRedshifts': {
                                                'Ampel z' : ['ampel_z'],
                                                'Ampel z group' : ['group_z_nbr'],
                                                'Ampel distance' :['ampel_dist'],
                                            },
                                            'T2RunSncosmo' : {
                                                'Peak B abs mag' : ['fit_metrics','restpeak_model_absmag_B'],
                                                'SALT2 X1' : ['sncosmo_result','paramdict','x1'],
                                                'SALT2 X1 (err)' : ['sncosmo_result','errors','x1'],
                                                'SALT2 Color' : ['sncosmo_result','paramdict','c'],
                                                'SALT2 Color (err)' : ['sncosmo_result','errors','c'],
                                                'Pulls around peak' : ['fit_metrics','absmean_peak_pull'],
                                                'Det. around peak' : ['fit_metrics','nbr_peak_pulls'],
                                            },
                                            'T2MatchBTS' : {
                                                'Type' : ['bts_type'],
                                                'IAU ID': ['bts_IAUID'],
                                            },
                                            'T2PhaseLimit' : {
                                                'Peak time' : ['t_peak'],
                                                'Est duration': ['t_masked_duration'],
                                                'Risetime g (0.5)': ['t_risedec','risedec_ztfg', 'rise_0.5'],
                                                'Falltime g (0.5)': ['t_risedec','risedec_ztfg', 'fall_0.5'],
                                                'Risetime r (0.5)': ['t_risedec','risedec_ztfr', 'rise_0.5'],
                                                'Falltime r (0.5)': ['t_risedec','risedec_ztfr', 'fall_0.5'],
                                            },
                                            
                                        },
                                        'transient_table_schema' : {
                                            
                                            
                                        },
                                        'include_stock' : True,
                                        'include_channels' : True,
                                        'fmt' : 'csv',
                                        'local_path' : workdir
                                    }

In [ ]:
t3p = ctx.new_context_unit(
    process_name = "TablePrint",
    unit = "T3Processor",
    execute = [
        {
            "unit": "T3ReviewUnitExecutor",
            "config": {
                "supply": {
                    "unit": "T3DefaultBufferSupplier",
                    "config": {
                        "select": {
                            "unit": "T3StockSelector",
                            "config": {"channel": channel}
                        },
                        "load": {
                            "unit": "T3SimpleDataLoader",
                            "config": {
                                "directives": ["STOCK", "T1", "T2DOC"],
                                "channel": channel
                            }
                        }
                     }
                },
                "stage" : {
                    "unit": "T3SimpleStager",
                    "config": {
                        "execute": [
                            {
                                "unit": "TransientTablePublisher",
                                "config": table_config
                            }
                        ]
                    }
                }
            }
        }
    ]
)

In [ ]:
t3p.run()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(workdir+'/TransientTable.csv')

In [ ]:
df